<a href="https://colab.research.google.com/github/Alcala09/bankruptcyNeuralNetwork/blob/main/UCLA_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#todo:
#logistic regression
#svm
#neural network
#knn

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.metrics import classification_report
from google.colab import drive
drive.mount('/content/drive')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # this is used for the plot the graph
import os
import seaborn as sns # used for plot interactive graph.
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
import sklearn.metrics.cluster as smc
from sklearn.model_selection import KFold


from matplotlib import pyplot
import itertools

%matplotlib inline

import random

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_orig = pd.read_csv("/content/drive/MyDrive/UCLA Project/Proj/american_bankruptcy.csv")
df  = df_orig.rename(columns={'X1': 'current assets', "X2": 'costs of good sold', 'X3': 'depreciation and amoritzation', 'X4': "EBITDA", "X5":"inventory", "X6": "net income", "X7": "total receivables", "X8":"market value", "X9": "net sales", "X10":"total assets", "X11": "total long-term debt", "X12":"EBIT", "X13":"gross profit", "X14":"total current liabilities", "X15":"retained earnings", "X16":"total revenue", "X17":"total liabilities", "X18": "total operatin expenses"})
df.head()

FileNotFoundError: ignored

In [ ]:
df.describe()

In [ ]:
df['status_label'] = df["status_label"].replace(["alive", "failed"], [1,0])

In [ ]:
df = df.sort_values(by = "status_label", ascending = True)
df = df.head(10000)

In [ ]:
df = df.drop(["year", "company_name"], axis = 1)


In [ ]:
from scipy import stats
def remove_rows_with_outliers(df, z_threshold=3, exclude_column=None):
    if exclude_column is not None:
        columns_to_check = df.columns[df.columns != exclude_column]
    else:
        columns_to_check = df.columns

    z_scores = np.abs(stats.zscore(df[columns_to_check]))
    row_outliers = np.any(z_scores > z_threshold, axis=1)
    filtered_df = df[~row_outliers]
    return filtered_df
#df = remove_rows_with_outliers(df, z_threshold=3, exclude_column = "status_label")

In [ ]:
import pandas as pd

def remove_rows_above_threshold(dataframe, column, threshold):
    # Copy the original DataFrame to avoid modifying it directly
    df = dataframe.copy()

    # Filter rows based on the specified column and threshold
    df = df[df[column] <= threshold]

    # Reset the index after removing rows
    df.reset_index(drop=True, inplace=True)

    return df

In [ ]:
# for i in range(2):
#   df = remove_rows_with_outliers(df, z_threshold=3, exclude_column = "status_label")

In [ ]:
# df = remove_rows_above_threshold(df, "current assets", 1000)
# df = remove_rows_above_threshold(df, "costs of good sold", 1000)
# df = remove_rows_above_threshold(df, "depreciation and amoritzation", 100)
# df = remove_rows_above_threshold(df, "inventory", 100)
# df = remove_rows_above_threshold(df, "total receivables", 200)
# df = remove_rows_above_threshold(df, "net sales", 1500)
# df = remove_rows_above_threshold(df, "total assets", 1500)
# df = remove_rows_above_threshold(df, "gross profit", 60000)
# df = remove_rows_above_threshold(df, "total current liabilities", 300)
# df = remove_rows_above_threshold(df, "total revenue", 2000)
# df = remove_rows_above_threshold(df, "total liabilities", 1000)
# df = remove_rows_above_threshold(df, "total operatin expenses", 1000)

# #market value

In [ ]:
df["status_label"].value_counts()

In [ ]:
df["status_label"].value_counts()

In [ ]:
#df['status_label'] = df["status_label"].replace([0, 1], [1,0])

In [ ]:
Y = df["status_label"].values
X = df.drop("status_label", axis  =1)
X = X.values

In [ ]:
import sys
assert sys.version_info >= (3, 5) # python>=3.5
import sklearn
assert sklearn.__version__ >= "0.20" # sklearn >= 0.20

import numpy as np #numerical package in python
import os
%matplotlib inline
import matplotlib.pyplot as plt #plotting package

#matplotlib magic for inline figures
%matplotlib inline
import matplotlib # plotting library
import matplotlib.pyplot as plt
corr_matrix = df.corr()
from pandas.plotting import scatter_matrix

ROOT_DIR = "."
IMAGES_PATH = os.path.join(ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

# scatter_matrix(df_orig, figsize=(30, 20))



# def save_fig(fig_name, tight_layout=True, fig_extension="png", resolution=300):
#     '''
#         plt.savefig wrapper. refer to
#         https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.savefig.html
#     '''
#     path = os.path.join(IMAGES_PATH, fig_name + "." + fig_extension)
#     print("Saving figure", fig_name)
#     if tight_layout:
#         plt.tight_layout()
#     plt.gca().set_aspect('equal')
#     plt.savefig(path, format=fig_extension, dpi=resolution)
# save_fig("scatter_matrix_plot")

In [ ]:
def save_fig(fig_name, tight_layout=True, fig_extension="png", resolution=300):
    '''
        plt.savefig wrapper. refer to
        https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.savefig.html
    '''
    path = os.path.join(IMAGES_PATH, fig_name + "." + fig_extension)
    print("Saving figure", fig_name)
    if tight_layout:
        plt.tight_layout()
    plt.gca().set_aspect('equal')
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
scaler = preprocessing.StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
train_X, test_X, train_Y, test_Y = train_test_split(X_scaled, Y, test_size = 0.2, random_state = 30)

In [ ]:
#it seems like the optimal lambda value is 65.
pd.DataFrame(train_X).describe()

In [ ]:
print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

In [ ]:
#logistic regression with no regularization
model = 0;
model = LogisticRegression()
model.fit(train_X, train_Y)
test_Y_hat = model.predict(test_X)

In [ ]:
test_Y_hat_cat = 1 * (test_Y_hat > 0.5)

In [ ]:
pd.DataFrame(test_Y_hat_cat).value_counts()

In [ ]:
pd.DataFrame(test_Y).value_counts()

In [ ]:
def print_4_metrics(target_test, predicted):
  print("%-12s %f" % ('Accuracy:', metrics.accuracy_score(target_test,predicted)))
  print("%-12s %f" % ('Precision:', metrics.precision_score(target_test, predicted,labels=None, pos_label=1, average='binary', sample_weight=None)))
  print("%-12s %f" % ('Recall:', metrics.recall_score(target_test, predicted,labels=None, pos_label=1, average='binary', sample_weight=None)))
  print("%-12s %f" % ('F1 Score:', metrics.f1_score(target_test, predicted,labels=None, pos_label=1, average='binary', sample_weight=None)))


In [ ]:
#logistic regression classification report before any regularization
print(classification_report(test_Y, test_Y_hat_cat))

In [ ]:
def draw_confusion_matrix(y, yhat, classes, title = "confusion matrix"):
    '''
        Draws a confusion matrix for the given target and predictions
        Adapted from scikit-learn and discussion example.
    '''
    plt.cla()
    plt.clf()
    matrix = confusion_matrix(y, yhat)
    plt.imshow(matrix, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title)
    plt.colorbar()
    num_classes = len(classes)
    plt.xticks(np.arange(num_classes), classes, rotation=90)
    plt.yticks(np.arange(num_classes), classes)

    fmt = 'd'
    thresh = matrix.max() / 2.
    for i, j in itertools.product(range(matrix.shape[0]), range(matrix.shape[1])):
        plt.text(j, i, format(matrix[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if matrix[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()


In [ ]:
# Plot ROC curve and report area under ROC
# use metrics.roc_curve(your y_test, predicted probabilities for y_test)
log_score = model.predict_proba(test_X)[:,1]
fpr_log_reg, tpr_log_reg, thresholds = metrics.roc_curve(test_Y,log_score)
print("Logistic Model Performance Results:\n")
pyplot.figure(1)
pyplot.plot(fpr_log_reg, tpr_log_reg, color='orange', lw=1)
pyplot.title("ROC curve: Log Reg before any regularization")
pyplot.xlabel('FPR')
pyplot.ylabel('TPR')
save_fig("roc curve log reg no reg")
aucroc = metrics.auc(fpr_log_reg, tpr_log_reg)
print('AUC of ROC: ', aucroc)
save_fig("confusion matrix log reg no reg")



In [ ]:
# report auc
# use metrics.auc(fpr, tpr)

draw_confusion_matrix(test_Y, test_Y_hat_cat, [1,0], "Confusion Matrix: logistic regression with no regularization")

In [ ]:
#logistic regression with L1(Lasso) regularization
lambdas = [70, 80, 90, 100]
for l in lambdas:
  model = 0;
  model = LogisticRegression(penalty = "l1", C = 1/l, solver = "liblinear")
  model.fit(train_X, train_Y)
  test_Y_hat = model.predict(test_X)
  test_Y_hat_cat = 1* (test_Y_hat) > 0.5
  print("lambda = " + str(l) + ":")
  print_4_metrics(test_Y, test_Y_hat_cat)
  draw_confusion_matrix(test_Y, test_Y_hat_cat, [1,0])
  print()
  print()

In [ ]:
#it appears that the optimal lambda is 65.
model = 0;
model = LogisticRegression(penalty = "l1", C = 1/65, solver = "liblinear")
model.fit(train_X, train_Y)
test_Y_hat = model.predict(test_X)
test_Y_hat_cat = 1* (test_Y_hat) > 0.5
print("lambda = " + str(65) + ":")
print_4_metrics(test_Y, test_Y_hat_cat)
print(classification_report(test_Y, test_Y_hat_cat))
draw_confusion_matrix(test_Y, test_Y_hat_cat, [1,0],"confusion matrix log reg l1 reg(lambda = 65)")

In [ ]:
log_score = model.predict_proba(test_X)[:,1]
fpr_log_reg, tpr_log_reg, thresholds = metrics.roc_curve(test_Y,log_score)
print("Logistic Model Performance Results:\n")
pyplot.figure(1)
pyplot.plot(fpr_log_reg, tpr_log_reg, color='orange', lw=1)
pyplot.title("ROC curve: Log Reg with l1 regularization")
pyplot.xlabel('FPR')
pyplot.ylabel('TPR')
save_fig("roc curve log reg l1 reg")
aucroc = metrics.auc(fpr_log_reg, tpr_log_reg)
print('AUC of ROC: ', aucroc)

#SVM

In [ ]:
model = 0;
model = SVC(probability = True)
model.fit(train_X, train_Y)
test_Y_hat = model.predict(test_X)
test_Y_hat_cat = 1*(test_Y_hat > 0.5)
svm_score = model.predict_proba(test_X)[:,1]
print_4_metrics(test_Y, test_Y_hat_cat)


In [ ]:
print(classification_report(test_Y, test_Y_hat_cat))

In [ ]:
draw_confusion_matrix(test_Y, test_Y_hat_cat, [1,0], "confusion matrix: SVM")

In [ ]:
svm_score = model.predict_proba(test_X)[:,1]
fpr_svm, tpr_svm, thresholds = metrics.roc_curve(test_Y,log_score)
print("SVM Performance Results:\n")
pyplot.figure(1)
pyplot.plot(fpr_svm, tpr_svm, color='orange', lw=1)
pyplot.title("ROC curve SVM: ")
pyplot.xlabel('FPR')
pyplot.ylabel('TPR')
save_fig("roc curve SVM l1 reg")
aucroc = metrics.auc(fpr_svm, tpr_svm)
print('AUC of ROC: ', aucroc)

#KNN

In [ ]:
k_values = [1,2,3,4,5,7,9,10,20,200]

for k in k_values:
  model = 0;
  model = KNeighborsClassifier(n_neighbors = k)
  model.fit(train_X, train_Y)
  test_Y_hat = model.predict(test_X)
  test_Y_hat_cat = 1* (test_Y_hat > .5)
  print("k = " + str(k))
  print_4_metrics(test_Y, test_Y_hat_cat)

In [ ]:
#it seems like the optimal k is 3.

In [ ]:
model = 0;
model = KNeighborsClassifier(n_neighbors = 3)
model.fit(train_X, train_Y)
test_Y_hat = model.predict(test_X)
test_Y_hat_cat = 1* (test_Y_hat > .5)
print_4_metrics(test_Y, test_Y_hat_cat)
print(classification_report(test_Y, test_Y_hat_cat))

In [ ]:
draw_confusion_matrix(test_Y, test_Y_hat_cat, [1,0], "confusion matrix KNN: ")

In [ ]:
knn_score = model.predict_proba(test_X)[:,1]
fpr_knn, tpr_knn, thresholds = metrics.roc_curve(test_Y,knn_score)
print("KNN Performance Results:\n")
pyplot.figure(1)
pyplot.plot(fpr_knn, tpr_knn, color='orange', lw=1)
pyplot.title("ROC curve: ")
pyplot.xlabel('FPR')
pyplot.ylabel('TPR')
save_fig("roc curve KNN")
aucroc = metrics.auc(fpr_knn, tpr_knn)
print('AUC of ROC: ', aucroc)

#neural network (FOR THE LOVE OF GOD DO NOT RUN THIS ON COLAB IT TAKES FOREVER)

In [ ]:

model = 0;
model = Sequential();
model.add(Dense(20, activation = "relu"))
model.add(Dense(20, activation = "relu"))
model.add(Dense(1, activation = "sigmoid"));
model.compile(loss = "binary_crossentropy")
model.fit(train_X, train_Y, epochs = 10000, verbose  = 0)
J = model.history.history["loss"]
plt.plot(J)

In [ ]:
test_Y_hat = model.predict(test_X)
test_Y_hat_cat = 1* (test_Y_hat > 0.5)
print(classification_report(test_Y, test_Y_hat_cat))
# 63/63 [==============================] - 0s 1ms/step
#               precision    recall  f1-score   support

#            0       0.55      0.83      0.66      1042
#            1       0.59      0.27      0.37       958

#     accuracy                           0.56      2000
#    macro avg       0.57      0.55      0.52      2000
# weighted avg       0.57      0.56      0.52      2000

#layer 1: 10 node relu
# 63/63 [==============================] - 0s 2ms/step
#               precision    recall  f1-score   support

#            0       0.72      0.52      0.61      1042
#            1       0.60      0.78      0.68       958

#     accuracy                           0.65      2000
#    macro avg       0.66      0.65      0.64      2000
# weighted avg       0.66      0.65      0.64      2000

#layer 1: 20 node relu
# 63/63 [==============================] - 0s 2ms/step
#               precision    recall  f1-score   support

#            0       0.70      0.66      0.68      1042
#            1       0.65      0.69      0.67       958

#     accuracy                           0.67      2000
#    macro avg       0.67      0.68      0.67      2000
# weighted avg       0.68      0.67      0.67      2000

#layer 1: 20 relu, layer 2: 20 relu(epochs = 2000)
# 63/63 [==============================] - 0s 2ms/step
#               precision    recall  f1-score   support

#            0       0.78      0.62      0.69      1042
#            1       0.66      0.81      0.73       958

#     accuracy                           0.71      2000
#    macro avg       0.72      0.72      0.71      2000
# weighted avg       0.73      0.71      0.71      2000
# 63/63 [==============================] - 0s 2ms/step
#               precision    recall  f1-score   support

#            0       0.73      0.70      0.71      1042
#            1       0.69      0.71      0.70       958

#     accuracy                           0.71      2000
#    macro avg       0.71      0.71      0.71      2000
# weighted avg       0.71      0.71      0.71      2000

In [ ]:
draw_confusion_matrix(test_Y, test_Y_hat_cat, [1,0], "confusion matrix: neural network")